In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import random

class SmartCaptionGenerator:
    def __init__(self):
        print("Loading GPT-Neo...")
        
        model_name = "EleutherAI/gpt-neo-125M"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        
        print("✓ Model ready!\n")
        
        #templates 
        self.templates = {
            'sunset': [
                "Chasing sunsets", "Golden hour magic", "Sky on fire",
                "Sunset state of mind", "Where the sky paints itself",
                "Every sunset is a promise", "Painting the sky",
                "Sunsets and silhouettes", "Watching the world turn gold"
            ],
            'beach': [
                "Beach therapy", "Salt in the air", "Ocean vibes",
                "Vitamin Sea", "Life's a beach", "Sandy toes, sun-kissed nose",
                "Ocean child", "Paradise found", "Beach please",
                "Seas the day", "High tides and good vibes"
            ],
            'festival': [
                "Festive feels", "Celebrating moments", "Festival vibes",
                "Lights, camera, celebration", "Party mode activated",
                "Festival season is here", "Good times and tan lines",
                "Celebrate every moment", "Making festival memories"
            ],
            'home': [
                "Home sweet home", "Cozy corner goals", "Making memories",
                "There's no place like home", "Home is where the heart is",
                "Cozy vibes only", "Home is my happy place",
                "Creating my sanctuary", "Living my best life at home"
            ],
            'diwali': [
                "Happy Diwali", "Festival of lights", "Diwali vibes",
                "Light over darkness", "Sparkle and shine", 
                "Diwali celebrations", "Festival of joy", "Lights, love, laughter"
            ],
            'food': [
                "Food coma incoming", "Good food, good mood", "Eating my feelings",
                "Food is my love language", "But first, food", "Foodie life",
                "Calories don't count today", "Food therapy", "Feast mode",
                "Eating good in the neighborhood", "Chef's kiss"
            ],
            'chai': [
                "Chai time is the best time", "But first, chai", "Chai lover",
                "Chai o'clock", "Everything stops for chai", "Tea time thoughts",
                "Steeped in tradition", "My daily dose of happiness"
            ],
            'coffee': [
                "But first, coffee", "Coffee is my love language", "Caffeinated vibes",
                "Espresso yourself", "Life happens, coffee helps",
                "Powered by caffeine", "Coffee and contemplation",
                "Brew-tiful morning", "Rise and grind", "Sip happens"
            ],
            'train': [
                "Journey mode ON", "Train travels and tales", "On the move",
                "Life is a journey", "All aboard", "Track side stories",
                "Railway adventures", "Going places", "Next station: happiness"
            ],
            'travel': [
                "Wanderlust", "Adventure awaits", "Going places",
                "Collect moments, not things", "Travel far, live fully",
                "Exploring the unknown", "Not all who wander are lost",
                "Journey over destination", "Adventure is out there",
                "Travel mode activated", "Passport ready", "World citizen"
            ],
            'friends': [
                "Squad goals", "Making memories", "Good times",
                "Friends who feel like family", "My tribe", "Circle of trust",
                "Here's to the nights we'll never forget", "Friendship goals",
                "Real ones", "Friends forever", "Good friends, good times"
            ],
            'birthday': [
                "Another trip around the sun", "Birthday vibes", "Leveling up",
                "Cheers to another year", "Making my wishes count",
                "Birthday mood", "Grateful for another year", "Aging like fine wine",
                "It's my party", "Blessed and grateful"
            ],
            'flowers': [
                "Bloom where you are planted", "Flower power", "Nature's beauty",
                "Stop and smell the flowers", "Petals and positivity",
                "In full bloom", "Botanical bliss", "Garden therapy",
                "Wild and free like flowers"
            ],
            'selfie': [
                "Feeling myself", "Confidence level: 100", "Self-love",
                "Just me being me", "Unapologetically me", "Self-care Sunday",
                "Living my truth", "Authentic vibes", "No filter needed",
                "Me, myself, and I", "Own your story"
            ],
            'nature': [
                "Nature therapy", "Into the wild", "Natural beauty",
                "Mother Nature at her finest", "Wild and free",
                "Earth's masterpiece", "Where I belong", "Nature heals",
                "Green therapy", "Back to basics"
            ],
            'gym': [
                "Gym time", "Earning that meal", "Fitness vibes",
                "No pain, no gain", "Sweat now, shine later",
                "Strong over skinny", "Workout mode", "Grind mode",
                "Building my empire one rep at a time"
            ],
            'rain': [
                "Rainy day vibes", "Dancing in the rain", "Petrichor lover",
                "Let it rain", "Monsoon magic", "Rain or shine",
                "Stormy weather, cozy feels", "Raindrop therapy"
            ],
            'music': [
                "Music is life", "Lost in music", "Vibing",
                "Music on, world off", "Lost in the rhythm",
                "Life is better with music", "My kind of therapy",
                "Turn up the volume", "Music speaks what cannot be expressed"
            ],
            'book': [
                "Book lover", "Reading mode ON", "Lost in pages",
                "One more chapter", "Book worm life", "Reading is dreaming",
                "Between the pages", "Literary escape", "My favorite adventure"
            ],
            'car': [
                "Road trip vibes", "Drive mode ON", "On the road",
                "Life is a highway", "Adventure on wheels",
                "Cruise control", "Miles of smiles", "Born to ride"
            ],
            'night': [
                "Night owl", "City lights", "Midnight vibes",
                "Under the stars", "Night time is the right time",
                "Moonlight magic", "After dark", "Nocturnal adventures"
            ],
            'morning': [
                "Rise and shine", "Morning vibes", "New day energy",
                "Fresh start", "Morning person", "Sunrise chaser",
                "Early bird gets the worm", "Good morning world",
                "New day, new possibilities"
            ],
            'work': [
                "Hustle mode ON", "Getting it done", "Work hard, dream big",
                "Boss mode", "Making it happen", "Grinding",
                "On my grind", "Building my empire", "Success in progress"
            ],
            'love': [
                "Love is in the air", "All you need is love",
                "Love wins", "Forever and always", "My happy place",
                "With you is my favorite place to be", "Love story",
                "Better together", "My person"
            ],
            'family': [
                "Family first", "Blood is thicker than water",
                "Family time is the best time", "My favorite people",
                "Home is where my family is", "Family over everything",
                "Making family memories", "Together is our favorite place"
            ],
            'fitness': [
                "Fitness is my therapy", "Strong is the new sexy",
                "Progress not perfection", "Fitness journey",
                "Train insane or remain the same", "Sweat equity",
                "Fit life", "Health is wealth"
            ],
            'wedding': [
                "Happily ever after", "The best day ever",
                "Just married", "Mr and Mrs", "Forever starts today",
                "Love and laughter", "Wedding bells", "Together forever"
            ],
            'graduation': [
                "The tassel was worth the hassle", "Graduated and educated",
                "New chapter unlocked", "Class dismissed forever",
                "Future starts now", "Diploma acquired", "Moving on and moving up"
            ],
            'yoga': [
                "Find your balance", "Namaste", "Yoga vibes",
                "Inhale confidence, exhale doubt", "Yoga is my therapy",
                "Bend so you don't break", "Peace, love, yoga"
            ],
            'pet': [
                "My furry friend", "Pet parent life", "Unconditional love",
                "Life is better with pets", "My best friend",
                "Pawsitively perfect", "Four-legged love", "My therapy animal"
            ],
            'fashion': [
                "Style is eternal", "Fashion fades, style is eternal",
                "Dressed to impress", "Fashion is my passion",
                "Outfit of the day", "Style game strong", "Fashion forward"
            ],
            'photography': [
                "Through my lens", "Captured moments",
                "Picture perfect", "Frozen in time", "Lens love",
                "Photography is my art", "Shutterbug life"
            ],
            'dance': [
                "Dance like nobody's watching", "Rhythm and moves",
                "Dance is my language", "Move to the beat",
                "Dancing through life", "Express through movement"
            ],
            'art': [
                "Art is therapy", "Creative soul", "Artistic vibes",
                "Creating magic", "Art speaks", "Lost in creation",
                "Creativity unleashed"
            ],
            'motivation': [
                "Make it happen", "Dream big, work hard",
                "Success is earned", "Keep pushing", "Never give up",
                "Believe in yourself", "You got this", "Limitless potential"
            ],
            'meditation': [
                "Inner peace", "Mindful moments", "Find your zen",
                "Meditation is my medication", "Breathe and let go",
                "Present moment awareness"
            ],
            'cooking': [
                "Chef mode activated", "Cooking with love",
                "Kitchen adventures", "Homemade happiness",
                "Cooking is my therapy", "From my kitchen to your heart"
            ],
            'party': [
                "Party time", "Let's celebrate", "Good vibes only",
                "Party animal", "Dance floor ready",
                "Making memories tonight", "Party mode activated"
            ]
        }
        
        # Keyword mapping for smart matching
        self.keyword_map = {
            'sunset': ['sunset', 'sunrise', 'dusk', 'golden hour', 'twilight', 'dawn', 'skyline'],
            'beach': ['beach', 'ocean', 'sea', 'shore', 'coast', 'waves', 'sand', 'seaside', 'coastline'],
            'chai': ['chai', 'tea', 'chay', 'masala chai'],
            'coffee': ['coffee', 'cafe', 'latte', 'cappuccino', 'espresso', 'mocha', 'americano'],
            'train': ['train', 'railway', 'metro', 'station', 'railroad', 'locomotive'],
            'travel': ['travel', 'trip', 'vacation', 'journey', 'adventure', 'wanderlust', 'explore', 'tour', 'backpack'],
            'friends': ['friend', 'squad', 'crew', 'gang', 'besties', 'frnds', 'buddy', 'pal', 'mate'],
            'birthday': ['birthday', 'bday', 'b-day', 'birth day', 'born day'],
            'flowers': ['flower', 'bloom', 'petal', 'garden', 'rose', 'lily', 'tulip', 'daisy', 'sunflower', 'bouquet'],
            'selfie': ['selfie', 'self', 'me', 'myself', 'self portrait'],
            'nature': ['nature', 'forest', 'mountain', 'hill', 'tree', 'woods', 'wilderness', 'outdoors', 'greenery'],
            'gym': ['gym', 'workout', 'fitness', 'exercise', 'training', 'weights', 'cardio'],
            'rain': ['rain', 'rainy', 'monsoon', 'drizzle', 'shower', 'storm', 'wet'],
            'music': ['music', 'song', 'concert', 'singer', 'band', 'melody', 'tune', 'beat', 'rhythm'],
            'book': ['book', 'reading', 'novel', 'library', 'literature', 'read', 'story', 'author'],
            'car': ['car', 'drive', 'road trip', 'driving', 'automobile', 'vehicle', 'ride'],
            'night': ['night', 'midnight', 'evening', 'dark', 'nocturnal', 'nighttime'],
            'morning': ['morning', 'dawn', 'breakfast', 'sunrise', 'am', 'early'],
            'food': ['food', 'eating', 'dinner', 'lunch', 'meal', 'hungry', 'feast', 'cuisine', 'dish', 'yummy'],
            'festival': ['festival', 'celebration', 'celebrate', 'party', 'fest', 'carnival'],
            'diwali': ['diwali', 'deepavali', 'deepawali'],
            'home': ['home', 'house', 'apartment', 'place', 'room', 'living'],
            'work': ['work', 'office', 'job', 'meeting', 'business', 'career', 'professional'],
            'love': ['love', 'romance', 'couple', 'relationship', 'valentine', 'heart', 'boyfriend', 'girlfriend', 'partner'],
            'family': ['family', 'parents', 'mom', 'dad', 'brother', 'sister', 'relatives', 'siblings'],
            'fitness': ['fitness', 'fit', 'health', 'healthy', 'strong', 'muscle', 'gains'],
            'wedding': ['wedding', 'marriage', 'bride', 'groom', 'married', 'ceremony', 'reception'],
            'graduation': ['graduation', 'graduate', 'degree', 'diploma', 'commencement', 'convocation'],
            'yoga': ['yoga', 'asana', 'meditation', 'mindfulness', 'zen', 'namaste'],
            'pet': ['pet', 'dog', 'cat', 'puppy', 'kitten', 'animal', 'fur baby', 'paw'],
            'fashion': ['fashion', 'style', 'outfit', 'clothes', 'dress', 'shopping', 'ootd'],
            'photography': ['photography', 'photo', 'camera', 'picture', 'photographer', 'lens', 'snap'],
            'dance': ['dance', 'dancing', 'dancer', 'choreography', 'performance'],
            'art': ['art', 'painting', 'drawing', 'artist', 'creative', 'artwork', 'design'],
            'motivation': ['motivation', 'motivate', 'inspire', 'goal', 'success', 'achievement'],
            'meditation': ['meditation', 'meditate', 'mindful', 'peace', 'calm', 'relax'],
            'cooking': ['cooking', 'cook', 'recipe', 'chef', 'kitchen', 'baking', 'bake'],
            'party': ['party', 'club', 'disco', 'nightclub', 'rave', 'clubbing']
        }
    
    def match_templates(self, description):
        """Match templates based on keywords"""
        desc_lower = description.lower()
        matched = []
        
        for category, keywords in self.keyword_map.items():
            if any(keyword in desc_lower for keyword in keywords):
                matched.extend(self.templates.get(category, []))
        
        return matched
    
    def generate_dynamic_captions(self, description):
        """Generate dynamic captions for ANY description"""
        # Extract key words from description
        words = description.split()
        key_words = [w for w in words if len(w) > 3]  # Focus on meaningful words
        
        dynamic_captions = []
        
        # Pattern 1: "[Description] vibes"
        dynamic_captions.append(f"{description.capitalize()} vibes")
        
        # Pattern 2: "Living for [key words]"
        if key_words:
            dynamic_captions.append(f"Living for {' and '.join(key_words[:2])}")
        
        # Pattern 3: "[Description] moments"
        dynamic_captions.append(f"{description.capitalize()} moments")
        
        # Pattern 4: Simple text
        dynamic_captions.append(f"{description.capitalize()}")
        
        # Pattern 5: "Perfect [description]"
        dynamic_captions.append(f"Perfect {description}")
        
        # Pattern 6: Generic good vibes
        dynamic_captions.extend([
            "Living my best life",
            "Making memories",
            "Good vibes only",
            "This is my happy place",
            "Moments like these"
        ])
        
        return dynamic_captions
    
    def generate(self, description, num_captions=3):
        """Generate captions - combines templates + dynamic generation"""
        
        # Try to match templates first
        template_captions = self.match_templates(description)
        
        # Generate dynamic captions for any description
        dynamic_captions = self.generate_dynamic_captions(description)
        
        # Combine both
        all_captions = template_captions + dynamic_captions
        
        # Remove duplicates while preserving order
        seen = set()
        unique_captions = []
        for cap in all_captions:
            if cap not in seen:
                seen.add(cap)
                unique_captions.append(cap)
        
        # Randomly select from available captions
        if len(unique_captions) >= num_captions:
            selected = random.sample(unique_captions, num_captions)
        else:
            selected = unique_captions + ["Living my best life"] * (num_captions - len(unique_captions))
        
        return selected[:num_captions]

# Main Program
if __name__ == "__main__":
    print("=" * 60)
    print("INSTAGRAM CAPTION GENERATOR")
    print("=" * 60)
    
    gen = SmartCaptionGenerator()
    
    while True:
        photo = input("\n Describe your photo (or 'quit'): ").strip()
        
        if photo.lower() in ['quit', 'q', 'exit']:
            print("\n Goodbye!")
            break
        
        if not photo:
            continue
        
        print(f"\n Captions for: '{photo}'\n")
        captions = gen.generate(photo, num_captions=3)
        
        for i, cap in enumerate(captions, 1):
            print(f"  {i}. {cap}")
        
        print()

INSTAGRAM CAPTION GENERATOR
Loading GPT-Neo...
✓ Model ready!




 Describe your photo (or 'quit'):  i'm cooking my fav dish



 Captions for: 'i'm cooking my fav dish'

  1. Foodie life
  2. I'm cooking my fav dish
  3. Good vibes only




 Describe your photo (or 'quit'):  festival celebration at home



 Captions for: 'festival celebration at home'

  1. There's no place like home
  2. Creating my sanctuary
  3. Self-love



...


 Captions for: 'prayers and blessings'

  1. Living my best life
  2. Moments like these
  3. Prayers and blessings moments




 Describe your photo (or 'quit'):  travelling with freinds



 Captions for: 'travelling with freinds'

  1. Living my best life
  2. Adventure awaits
  3. Not all who wander are lost




 Describe your photo (or 'quit'):  quit



 Goodbye!


The model i used EleutherAI/gpt-neo-125M
The system first tries to match the photo description with predefined templates
If no perfect template is found, it generates dynamic captions using simple sentence patterns
Each time when we run the code , it gives different captions each time , even for the same input